<a href="https://colab.research.google.com/github/AlexBB999/Thinkful/blob/master/31_4_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ASSIGNMENT ANSWERS AT BOTTOM**

##**Assignments**

In the following exercises, you'll be working with the code snippet below:

%%timeit
x = da.random.random((10000, 10000), chunks=(1000, 1000))

y = x + x.T

z = y[::2, 5000:].mean(axis=1)

z.compute()


To complete this assignment, create a Jupyter Notebook containing your solutions to the following tasks and submit as a link on Github.

Change the code above by setting chunks=(250, 250).

 How long does it take to run?

Now, set the parameter to chunks=(500, 500).

 How long does it take to run? 

Does this one or the previous one run quickly? Why?

In [2]:
!pip install --upgrade "dask[complete]"

     |████████████████████████████████| 808kB 4.9MB/s 
     |████████████████████████████████| 634kB 28.5MB/s 
     |████████████████████████████████| 501kB 32.1MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=46eb8fa77b9c4a2d1264573241fcaecec3688110bb1b7f06d0a0b1b6412328d6
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=180a3f105624971bf17b8380f1a813f04a97ef9f5e76222e00942e398c832913
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for tornado: filename=tornado-6.0.4-cp36-cp36m-linux_x86_64.whl size=427638 sha256=1148753ffc229566aa06c5c8f3319b8f5350bd36a269eebda885cf01b61361b2
  Stored in directory: /root/.cache/pip/wheels/93/84/2f/409c7b2bb3afc3aa727f7ee8787975e0793

In [3]:
pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.2.1)


In [1]:
import warnings
warnings.filterwarnings("ignore")

from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:37361 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


##**Using Dask as if it's NumPy**
We'll be using Dask arrays by creating random arrays and do mathematical calculations on them. 

We also do the same thing using NumPy arrays and compare the run times.

###**Creating a random array**
In order to use Dask arrays, we need to import it as follows:

In [0]:
import dask.array as da
import numpy as np

Below we create a 10000x10000 array of random numbers.

 Note that we set chunks parameter to (1000, 1000).
 
 This is something different than what we normally do when generating random arrays with NumPy.
 
By setting chunks, we tell Dask that it should represent as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly).

In our case, there will be 100 numpy arrays of size 1000x1000.

What we do below is:

**We first create a random Dask array of size 10000X10000.**

**Then we add this array to its transpose**.

**Last, we filter the resulting array and calculate its mean**.

As usual, we call .compute() to make Dask evaluate the results.

Note that we calculate the run time of the following cell using jupyter notebook's **magic command %%time**

In [5]:
%%time
x = da.random.random((10000, 10000), chunks=(1000, 1000))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 372 ms, sys: 69.4 ms, total: 441 ms
Wall time: 1.56 s


**Now, let's do the same thing using NumPy arrays:**

In [4]:
%%time
x = np.random.random((10000, 10000))
y = x + x.T
z = y[::2, 5000:].mean(axis=1)

CPU times: user 1.61 s, sys: 745 ms, total: 2.35 s
Wall time: 2.26 s


**CHUNK METHOD MUCH FASTER**

**/////////////////////////////////////////////////////////////////**

##**Persisting data in memory -- IF POSSIBLE**

**First, we make our computations without persisting the array**

In [0]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))

In [7]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 265 ms, sys: 54.7 ms, total: 320 ms
Wall time: 1.42 s


Now, we do the same thing this time **after persisting our array into the memory**

In [8]:
x = da.random.random((10000, 10000), chunks=(1000, 1000))
# This persists the x array into the memory
x.persist()

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

**And we run the same computations above after persisting the array into the memory**

In [9]:
%%time
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z.compute()

CPU times: user 200 ms, sys: 30 ms, total: 230 ms
Wall time: 856 ms


**FASTER BUT NOT HALF THE TIME**

---



**////////////////////////////////////////////////////////////////////////////////**

In [10]:
%%timeit x = da.random.random((10000, 10000), chunks=(1000, 1000))

y = x + x.T

z = y[::2, 5000:].mean(axis=1)

z.compute()

1 loop, best of 3: 1.41 s per loop


**Change the code above by setting chunks=(250, 250).**

In [11]:
%%timeit x = da.random.random((10000, 10000), chunks=(250, 250))

y = x + x.T

z = y[::2, 5000:].mean(axis=1)

z.compute()

1 loop, best of 3: 5.23 s per loop


**MUCH SLOWER**

**Now, set the parameter to chunks=(500, 500).**

In [12]:
%%timeit x = da.random.random((10000, 10000), chunks=(500, 500))

y = x + x.T

z = y[::2, 5000:].mean(axis=1)

z.compute()

1 loop, best of 3: 2.28 s per loop


**MUCH FASTER**

**NOT ASSIGNED BUT TRYING CHUNK SIZE=2000**

In [14]:
%%timeit x = da.random.random((10000, 10000), chunks=(2000, 2000))

y = x + x.T

z = y[::2, 5000:].mean(axis=1)

z.compute()

1 loop, best of 3: 1.37 s per loop


**EVEN FASTER**